## Dashboard Interactivo del Modelo Mundell-Fleming

Este notebook contiene el código para generar un dashboard interactivo del modelo Mundell-Fleming. Permite visualizar los efectos de políticas fiscales, monetarias y shocks cambiarios en una economía abierta con tipo de cambio flexible.